<a href="https://colab.research.google.com/github/AMDonati/RL-NLP/blob/experiments_low_scale/truncated_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Truncated action space
The goal of this notebook is to test if the truncation of the action by a pretrained language model ease the learning process


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%load_ext tensorboard

In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [0]:
#!git clone https://github.com/AMDonati/RL-NLP.git

In [5]:
cd RL-NLP/

/content/drive/My Drive/RL-NLP


In [0]:
!git fetch

In [7]:
!git checkout experiments_low_scale

M	src/scripts/run.py
Already on 'experiments_low_scale'
Your branch is up to date with 'origin/experiments_low_scale'.


In [0]:
#rm src/scripts/run.py

In [0]:
#!git reset --hard

In [0]:
#cd ..

In [11]:
! git pull 

Already up to date.


In [0]:
#!pip install -r requirements.txt

In [13]:
cd src

/content/drive/My Drive/RL-NLP/src


# Run


##Arguments to run the script:

**Generic arguments:**
*   data_path: str: path where h5 files are located
*   out_path: str: path where the output files will be written
* max_len: int: maximum length of the target sentences and the sentences produced by the agent 
* logger_level:str : level of the logger
* num_episodes_train:int: number of episodes for training
*  num_episodes_test:int: number of episodes for testing
* log_interval:int: interval of episodes for logging
* reward:str: type of reward [levenshtein: normalized levenshtein distance, levenshtein_: levenshtein distance] 
* model:str: predefined model for the policy [lstm: for text and image, lstm_word: for text(lm)]
* update_every:int: call the learn function every update_every
* debug:int: if specified the number of images used for training will be the value of debug
* agent:str: type of RL algorithm used (PPO, REINFORCE)

**Pretraining:**

* pretrain:int: pretrain the model to get a learned language model using the RL method
* pretrained_path:str: if specified, the language model will be loaded from this path to truncate the action space
* num_truncated:int: number of top words of the language model to truncate the action space

**RL Algorithm:**

PPO
* eps_clip:float float: epsilon clip 
* K_epochs:int: number of epochs of learning for each call to update
* entropy_coeff:float: coefficient of the entropy bonus of the loss

REINFORCE
* lr:float: learning rate

**Image**
* kernel_size:int: the size of the kernel used for convolution on images features from VGG
* stride:int: stride for the convolution (no maxpooling)
* num_filters:int: number of filters

**Text**
* word_emb_size:int: size of the embedding of the words
* hidden_size:int: size of the hidden state

In [0]:
data_path= "../data/"
out_path= "../output/truncated_0"
max_len= 5
logger_level= "DEBUG" 
num_episodes_train=2000
log_interval= 1 
reward = "levenshtein_"
model="lstm"
update_every=20
K_epochs= 10 
entropy_coeff=0.01 
eps_clip =0.02 
pretrain=0 
debug= 1
num_episodes_test =100
agent ="PPO"
hidden_size=24
num_truncated=10

## Train the language model


In [16]:
!python -m scripts.run -data_path $data_path -out_path $out_path -max_len 10 -logger_level $logger_level -num_episodes_train 2000 -log_interval $log_interval -reward $reward -model lstm_word -update_every $update_every -K_epochs $K_epochs -entropy_coeff $entropy_coeff -eps_clip $eps_clip -pretrain 1 -debug 0 -num_episodes_test $num_episodes_test -agent $agent -hidden_size $hidden_size

2020-06-10 19:56:27.239026: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


In [0]:
#save the last folder, the one of the language model
LM=!ls -dt -- $out_path/experiments/pretrain/* | head -n1
LM=LM[0]

In [0]:
pretrained_path="{}/model.pth".format(LM)

## One image experiment: Truncated vs Baseline

In [0]:
num_episodes_train=2500
debug=1
max_len=5

### Train the agent from scratch with the truncated action space given by the language model

In [0]:
train_cmd_truncated="python -m scripts.run -data_path {} -out_path {} -max_len {} -logger_level {} -num_episodes_train {} -log_interval {} -reward {} -model {} -update_every {} -K_epochs {} -entropy_coeff {} -eps_clip {} -pretrain {} -debug {} -num_episodes_test {} -agent {} -hidden_size {} -pretrained_path {} -num_truncated {}".format(data_path,out_path,max_len,logger_level,num_episodes_train,log_interval,reward,model,update_every,K_epochs,entropy_coeff,eps_clip,pretrain,debug,num_episodes_test,agent,hidden_size,pretrained_path,num_truncated)

In [0]:
for i in range(3):
  !$train_cmd_truncated

2020-06-10 20:04:30.620489: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-06-10 20:06:35.428104: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1


### Train the baseline agent from scratch with no language model




In [0]:
train_cmd_baseline="python -m scripts.run -data_path {} -out_path {} -max_len {} -logger_level {} -num_episodes_train {} -log_interval {} -reward {} -model {} -update_every {} -K_epochs {} -entropy_coeff {} -eps_clip {} -pretrain {} -debug {} -num_episodes_test {} -agent {} -hidden_size {} ".format(data_path,out_path,max_len,logger_level,num_episodes_train,log_interval,reward,model,update_every,K_epochs,entropy_coeff,eps_clip,pretrain,debug,num_episodes_test,agent,hidden_size)

In [0]:
for i in range(3):
  !$train_cmd_baseline

# Visualize results

In [0]:
%tensorboard --logdir $out_path